<a href="https://colab.research.google.com/github/paryoja/deeplearningstudy/blob/master/ReinforcementLearning/gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

먼저 gym을 설치합니다

출처 https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t#scrollTo=pdb2JwZy4jGj&forceEdit=true&offline=true&sandboxMode=true

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

팩맨 설치

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Helper function들 

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

from tensorflow import keras
import matplotlib.pyplot as plt

가상 화면을 하나 만듭니다

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

show_video는 저장된 비디오를 보여주는 역할
wrap_env는 플레이를 비디오로 저장하는 역할

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""



def show_video():
    html=[]
    mp4list = glob.glob('video/*.mp4')
    print(len(mp4list))
    if len(mp4list) > 0:
        for mp4 in mp4list:
            video = io.open(mp4, 'r+b').read()
            encoded = base64.b64encode(video)

            html.append(
                '''<video alt="test" autoplay 
                        controls style="height: 400px;">
                        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                     </video>'''.format(encoded.decode('ascii'))
            )

        ipythondisplay.display(HTML(data='\n'.join(html)))

    else: 
        print("Could not find video")


def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

# Pacman!

In [0]:
env = wrap_env(gym.make("MsPacman-v0"))

In [0]:
#check out the pacman action space!
print(env.action_space)

policy를 정해주는 네트워크 구성

In [0]:
# 일단 입력은 observation -> 출력은 action인 network 설정
import random

class policy_network:
    def __init__(self):
        self.ph = tf.placeholder(tf.float32, shape=[None, 210, 160, 3])
        self.filter_1 = tf.Variable(tf.random_normal([3, 3, 3, 10]))
        self.filter_2 = tf.Variable(tf.random_normal([3, 3, 10, 10]))
        self.filter_3 = tf.Variable(tf.random_normal([3, 3, 10, 10]))
        self.W = tf.Variable(tf.random_normal([328640, env.action_space.n],
                                             stddev=(1.0 / (328640.0 * env.action_space.n))), 
                             dtype=tf.float32)
        self.b = tf.Variable(env.action_space.n, dtype=tf.float32)
        
        self.ph_a = tf.placeholder(tf.int32, shape=[None, 1], name='action')
        self.reward = tf.placeholder(tf.float32, shape=[None, 1], name='reward')
        
    def build(self):
        # 입력은 -128 을 한다음 128로 나눠서 -1~1 사이에 위치하도록 한다
        norm_x = (self.ph - 128) / 128
        # [bs, 208, 158, 10]
        
        x = tf.nn.conv2d(norm_x, self.filter_1, [1, 1, 1, 1], "VALID")
        x = tf.nn.relu(x)
        x = tf.nn.conv2d(x, self.filter_2, [1, 1, 1, 1], "SAME")
        x = tf.nn.relu(x)
        x = tf.nn.conv2d(x, self.filter_3, [1, 1, 1, 1], "SAME")
        x = tf.nn.relu(x)
        x = tf.reshape(x, [-1, 328640])
        self.x = tf.matmul(x, self.W) + self.b
        self.sm = tf.nn.softmax(self.x)
        
        logit = tf.one_hot(self.ph_a, env.action_space.n)
        self.loss = -tf.reduce_sum(tf.log(self.sm * logit) * self.reward)
        
        opt = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        
        self.train_op = opt.minimize(self.loss)
    
    def get_action(self, sess, observation):
        
        # 확률을 예측하고
        prob = sess.run(self.sm, feed_dict={self.ph: observation})
        # print(prob)
        # 확률을 따라 action을 선택한다
        return self.choice(prob[0]), prob
        # [0.01, 0.09, 0.15, .... ]
        # 0 ~ 0.01 -> 1 action
        # 0.0100000001 -> ~ 0.10 -> 2 action
        # 0.1000000001 ~ 0.25 -> 3 action
    
    def update_param(self, sess, observation, reward, action):
        sess.run(self.train_op, feed_dict={
            self.ph: observation,
            self.ph_a: action,
            self.reward: reward,
        })
    
    def choice(self, prob):
        r = random.random()
        
        for i, p in enumerate(prob):
            r -= p
            
            if r < 0:
                return i
        return i
        

value function의 네트워크 구성

In [0]:
class value_network:
    def __init__(self):
        self.ph = tf.placeholder(tf.float32, shape=[None, 210, 160, 3])
        self.filter_1 = tf.Variable(tf.random_normal([3, 3, 3, 10]))
        self.filter_2 = tf.Variable(tf.random_normal([3, 3, 10, 10]))
        self.filter_3 = tf.Variable(tf.random_normal([3, 3, 10, 10]))
        self.W = tf.Variable(tf.random_normal([328640, 1],
                                             stddev=(1.0 / 328640.0)), 
                             dtype=tf.float32)
        self.b = tf.Variable(1, dtype=tf.float32)
        
        self.ph_a = tf.placeholder(tf.int32, shape=[None, 1], name='action')
        self.reward = tf.placeholder(tf.float32, shape=[None, 1], name='reward')
        
    def build(self):
        # 입력은 -128 을 한다음 128로 나눠서 -1~1 사이에 위치하도록 한다
        norm_x = (self.ph - 128) / 128
        # [bs, 208, 158, 10]
        
        x = tf.nn.conv2d(norm_x, self.filter_1, [1, 1, 1, 1], "VALID")
        x = tf.nn.relu(x)
        x = tf.nn.conv2d(x, self.filter_2, [1, 1, 1, 1], "SAME")
        x = tf.nn.relu(x)
        x = tf.nn.conv2d(x, self.filter_3, [1, 1, 1, 1], "SAME")
        x = tf.nn.relu(x)
        x = tf.reshape(x, [-1, 328640])
        self.x = tf.matmul(x, self.W) + self.b
        self.sm = tf.nn.softmax(self.x)
        
        logit = tf.one_hot(self.ph_a, env.action_space.n)
        self.loss = -tf.reduce_sum(tf.log(self.sm * logit) * self.reward)
        
        opt = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        
        self.train_op = opt.minimize(self.loss)
    
    def get_action(self, sess, observation):
        
        # 확률을 예측하고
        prob = sess.run(self.sm, feed_dict={self.ph: observation})
        # print(prob)
        # 확률을 따라 action을 선택한다
        return self.choice(prob[0]), prob
        # [0.01, 0.09, 0.15, .... ]
        # 0 ~ 0.01 -> 1 action
        # 0.0100000001 -> ~ 0.10 -> 2 action
        # 0.1000000001 ~ 0.25 -> 3 action
    
    def update_param(self, sess, observation, reward, action):
        sess.run(self.train_op, feed_dict={
            self.ph: observation,
            self.ph_a: action,
            self.reward: reward,
        })
    
    def choice(self, prob):
        r = random.random()
        
        for i, p in enumerate(prob):
            r -= p
            
            if r < 0:
                return i
        return i
        

In [0]:
d = policy_network()

sess = tf.Session()

d.build()
replay = []
avg_replay = []
final_reward = []

sess.run(tf.global_variables_initializer())

학습해보자

In [0]:
env = wrap_env(gym.make("MsPacman-v0"))

episode = 5


gamma = 0.99

total_length = 0
avg_reward = 0

for ep in range(episode):
    play = []
    observation = env.reset()
    
    prev_live = 0
    action = 0
    
    stay_action = 0
    
    while True:

        #env.render()

        # your agent goes here
        # action = env.action_space.sample() 
        if stay_action == 0:
            if random.random() > 0.5:
                action, _ = d.get_action(sess, [observation])
            else:
                action = env.action_space.sample()
            stay_action = 3
        else:
            stay_action -= 1
        
        # print(action)
        prev_ob = observation
        observation, reward, done, info = env.step(action)
        
        if prev_live != info['ale.lives'] and info['ale.lives'] != 0:
            for i in range(60):
                # 60 frame (2초) 은 의미 없어서 스킵
                env.step(env.action_space.sample())
            prev_live = info['ale.lives']
            
            
     
        
        # print("action {}, reward {}".format(action, reward))
        play.append([prev_ob, action, observation, reward, info])

        if done: 
          break;
    replay.append(play)
    print(info)    
    
    
    print(np.shape(play[0][0]))
    print(len(play))
    
    total_length += len(play)
    
    moving_average = -100 # 죽기 직전 행동은 안좋은 것으로 세팅
    total_reward = 0
    prev_lives = 0
    # 거꾸로 보면서 누적 reward 계산
    for p in reversed(play):
        # base function 을 -10으로 세팅
        if p[4]['ale.lives'] != prev_lives:
            moving_average = -100
            prev_lives = p[4]['ale.lives']
        
        moving_average = gamma * moving_average + (p[3] - 10)
        total_reward += p[3]
        p.append(moving_average)
    
    final_reward.append(total_reward)
    print(total_reward)
    if len(replay) > 100:
        del replay[0]
        del final_reward[0]
    avg_reward += total_reward / episode
avg_replay.append((total_length / episode, avg_reward))
#     print(play[0][1])
#     print(play[0][2])
env.close()
show_video()



In [0]:
import time
start = time.time()

for i in range(len(replay)):
    minibatch_ob = []
    minibatch_reward = []
    minibatch_action = []
    total_reward = []
    
    for k in range(100):
        r = random.randint(0, len(replay) - 1)
        s = random.randint(0, len(replay[r]) - 1)
    
        x = replay[r][s]
        minibatch_ob.append(x[2])
        minibatch_reward.append(x[-1])
        minibatch_action.append(x[1])
        
        total_reward.append(final_reward[r])
    # print(x[1], x[-1])
    d.update_param(sess, minibatch_ob, np.reshape(minibatch_reward, [-1,1]), np.reshape(minibatch_action, [-1, 1]))    
    
    if i % 10 == 9:
        print(i)
        end = time.time()
        print(end - start)
        start = end

In [0]:
print(avg_replay)